In [184]:
function kinetic(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    dphi=n*(circshift(phi,1)-phi)
    (dphi'*dphi)/(n*norm)
end

kinetic (generic function with 1 method)

In [187]:
function dkinetic_dphistar(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    dphi=n*(circshift(phi,1)-phi)
    ddphi=n*(circshift(dphi,-1)-dphi)
    (ddphi)/(n*norm)-(kinetic(phi,V,eta)/(norm*n))*phi
end

dkinetic_dphistar (generic function with 1 method)

In [185]:
function potential(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    (phi'*(V.*phi))/(n*norm)
end

potential (generic function with 1 method)

In [188]:
function dpotential_dphistar(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    (V.*phi)/(n*norm)-(potential(phi,V,eta)/(norm*n))*phi
end

dpotential_dphistar (generic function with 1 method)

In [186]:
function interaction(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    phisq=(phi.*phi)
    ((eta/2)*(phisq)'*(phisq))/(n*norm)
end

interaction (generic function with 1 method)

In [190]:
function dinteraction_dphistar(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    phisq=(phi.*phi)
    eta*(conjugate(phi).*phisq)/(n*norm)-2*(interaction(phi,V,eta)/(norm*n))*phi
end

dinteraction_dphistar (generic function with 1 method)

In [ ]:
function d2kinetic_dphistar_dphistar(phi,V,eta)
    (kinetic(phi,V,eta)/(norm*n))^2*phi*transpose(phi)
end

The following is only roughly right -- trying to think clearly about the clean way to set this up.

In [192]:
function d1kinetic_dphistar_dphi(phi,V,eta)
    n=length(phi)
    norm=(phi)'*(phi)/n
    result=2*(kinetic(phi,V,eta)/(norm*n)^2)*phi*(phi')
    for j in 1:n
        result[[j,j]]+=2*(kinetic(phi,V,eta)/(norm*n)^2) -2/n
    end
    for j in 1:n-1
        result[[j,j+1]]+=1/n
        result[[j+1,j]]+=1/n
    end
    result
end

search: transpose TransparentColor



```
transpose(A)
```

Lazy transpose. Mutating the returned object should appropriately mutate `A`. Often, but not always, yields `Transpose(A)`, where `Transpose` is a lazy transpose wrapper. Note that this operation is recursive.

This operation is intended for linear algebra usage - for general data manipulation see [`permutedims`](@ref Base.permutedims), which is non-recursive.

# Examples

```jldoctest
julia> A = [3 2; 0 0]
2×2 Matrix{Int64}:
 3  2
 0  0

julia> B = transpose(A)
2×2 transpose(::Matrix{Int64}) with eltype Int64:
 3  0
 2  0

julia> B isa Transpose
true

julia> transpose(B) === A # the transpose of a transpose unwraps the parent
true

julia> Transpose(B) # however, the constructor always wraps its argument
2×2 transpose(transpose(::Matrix{Int64})) with eltype Int64:
 3  2
 0  0

julia> B[1,2] = 4; # modifying B will modify A automatically

julia> A
2×2 Matrix{Int64}:
 3  2
 4  0
```

For complex matrices, the `adjoint` operation is equivalent to a conjugate-transpose.

```jldoctest
julia> A = reshape([Complex(x, x) for x in 1:4], 2, 2)
2×2 Matrix{Complex{Int64}}:
 1+1im  3+3im
 2+2im  4+4im

julia> adjoint(A) == conj(transpose(A))
true
```

The `transpose` of an `AbstractVector` is a row-vector:

```jldoctest
julia> v = [1,2,3]
3-element Vector{Int64}:
 1
 2
 3

julia> transpose(v) # returns a row-vector
1×3 transpose(::Vector{Int64}) with eltype Int64:
 1  2  3

julia> transpose(v) * v # compute the dot product
14
```

For a matrix of matrices, the individual blocks are recursively operated on:

```jldoctest
julia> C = [1 3; 2 4]
2×2 Matrix{Int64}:
 1  3
 2  4

julia> D = reshape([C, 2C, 3C, 4C], 2, 2) # construct a block matrix
2×2 Matrix{Matrix{Int64}}:
 [1 3; 2 4]  [3 9; 6 12]
 [2 6; 4 8]  [4 12; 8 16]

julia> transpose(D) # blocks are recursively transposed
2×2 transpose(::Matrix{Matrix{Int64}}) with eltype Transpose{Int64, Matrix{Int64}}:
 [1 2; 3 4]   [2 4; 6 8]
 [3 6; 9 12]  [4 8; 12 16]
```

---

```
transpose(F::Factorization)
```

Lazy transpose of the factorization `F`. By default, returns a [`TransposeFactorization`](@ref), except for `Factorization`s with real `eltype`, in which case returns an [`AdjointFactorization`](@ref).


In [ ]:
function dbarekin_dphistar1_update!(result,phi,scale)
    n=length(phi)
    result[n]+=(2*n*phi[n]-n*phi[1])*scale
    result[1]+=-n*phi[n]*scale
    for i in 1:n-1 #interior points
        result[i]+=2*n*phi[i]*scale
        result[i]+=-n*phi[i+1]*scale
        result[i+1]+=-n*phi[i]*scale
    end
    return result
end